## JAMES' AMES IOWA DATASET REGRESSION CHALLENGE

#### Objective

The objective of this project is to create a regression model based on the Ames Housing Dataset. This model will predict the price of a house at sale.

The project will focus on sharpening the following skills:

- Refining models over time
- Use of train-test split
- Use of Cross-validation
- Using data with unknown values for the target to simulate the modeling process


In [501]:
#libraries
import numpy as np
import pandas as pd
import seaborn as sns

#sklearn
from sklearn.linear_model import LogisticRegression, ElasticNet, ElasticNetCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

import matplotlib.pyplot as plt
%matplotlib inline

### EDA

In [502]:
# Get data
test_data = pd.read_csv("./datasets/test.csv")
train_data = pd.read_csv("./datasets/train.csv")

In [503]:
# Make into Dataframes
df_test = pd.DataFrame(test_data)
test_data.head()

In [504]:
# Make train into Data frame
df_train = pd.DataFrame(train_data)
train_data.head()

In [505]:
print("Test data shape: ", df_test.shape)
print("Train data shape: ", df_train.shape)


In [506]:
null_columns = df_train.isnull().sum() > 0
df_train.loc[:,null_columns].head()

In [507]:
df_train.info()

In [508]:
# Lot footage is a float with missing values, we can add impute some data for these
df_train['Lot Frontage'].plot.hist();
df_train['Lot Frontage'].describe()

In [509]:
df_train['Lot Frontage'].fillna(69.1, inplace=True)

df_train['Lot Frontage'].describe()
# Does not affect stats greatly by using mean for missing values

In [510]:
df_test['Lot Frontage'].fillna(69.6, inplace=True)
df_test['Lot Frontage'].describe()
# Does not affect stats greatly by using mean for missing values

In [511]:
df_train['Mas Vnr Area'].fillna(0.0, inplace=True)
# For another float with missing data, set values to 0, same as if it there was None
# Only 30 homes like this so won't make a huge impact

In [512]:
# Take a closer look at the missing basement data
# Will fill in missing categorical data to NA since bsmt sq ft is equal to 0
bsmt_list_cat = ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']
df_train.fillna({x:'NA' for x in bsmt_list_cat}, inplace= True) 

In [513]:
df_train.isnull().sum().sort_values(ascending= False).head(10)

In [514]:
#Pool is the highest null still
# All null quality are just have no pool so will set to NA
df_train['Pool QC'].fillna('NA', inplace= True)

In [515]:
df_train.isnull().sum().sort_values(ascending= False).head(10)

In [516]:
df_train['Misc Feature'].value_counts()
# just a random feature that is categorical so will set to NA
df_train['Misc Feature'].fillna('NA', inplace= True)

In [517]:
# If null value, will assume there is no alley
df_train['Alley'].fillna('NA', inplace= True)

In [518]:
# If null value, will assume there is no fence
df_train['Fence'].fillna('NA', inplace= True)

In [519]:
df_train.isnull().sum().sort_values(ascending= False).head(10)

In [520]:
df_train.groupby('Fireplaces').size()

In [521]:
# If null value, will assume there is no fireplace
df_train['Fireplace Qu'].fillna('NA', inplace= True)

In [522]:
# Look to see if garage cars being at 0 corresponds with null values, looks like it does
df_train.loc[df_train["Garage Cars"] ==0 ,df_train.columns.str.contains('Garage')].head()

In [523]:
# We can fill the categorical missing data with NA, but year built is a float
df_train['Garage Yr Blt'].describe()
# uh oh we have a garage built in the future 2207, I think that really should be 2007

In [524]:
df_train[df_train['Garage Yr Blt'] == 2207]

In [525]:
df_train.at[1699, 'Garage Yr Blt'] = 2007

In [526]:
df_train[df_train['Garage Yr Blt'] == 2207]

In [527]:
df_train['Garage Yr Blt'].plot.hist(bins=10)
df_train['Year Built'].plot.hist(bins=20)

In [528]:
df_train['Garage Yr Blt'].fillna(df_train['Year Built'], inplace=True)

# use year built when no garage present

In [529]:
# fill in remaining categorical garage data with NA
garage_list = list(df_train.loc[:,df_train.columns.str.contains('Garage')].columns)
[df_train[x].fillna('NA', inplace= True) for x in garage_list]

In [530]:
df_train.isnull().sum().sort_values(ascending= False).head(10)

In [531]:
# Fill in missing Masonary Veener Type with None
df_train['Mas Vnr Type'].fillna('None', inplace= True)

In [532]:
df_train.isnull().sum().sort_values(ascending= False).head(10)

In [533]:
# We will just drop the remaining
df_train.dropna(inplace= True)
df_train.isnull().sum().sort_values(ascending= False).head(10)

In [534]:
# Lets take a close look at sale price
df_train['SalePrice'].describe()

In [536]:
fig, ax = plt.subplots(figsize = (10,6))
salepriceplot = sns.distplot(df_train['SalePrice'], kde=True,axlabel='Sale Price of Home', hue="Yr Sold")
sns.despine(ax=ax, offset=10)
fig = salepriceplot.get_figure()
fig.savefig("SalePrice.png")

In [ ]:
Parcel_id = df_train['PID']
df_train.drop(columns=['Id', 'PID'], inplace=True)

In [ ]:
# Isolate correlation with respect to price 
corr_col = df_train.corrwith(df_train['SalePrice']).sort_values(ascending=False)[1:].to_frame()
fig2, ax2 = plt.subplots(figsize = (10,16))
corrplot = sns.heatmap(corr_col, cmap= 'coolwarm', annot=True, vmin=-1, vmax=1);
fig2 = corrplot.get_figure()
fig2.savefig("CorrPlot.png")

All EDA below here was done after initial runs to updated items I noticed after different iterations:

In [ ]:
# Check saleprice against highly correlated value
sns.lmplot(x='Gr Liv Area', y='SalePrice', data=df_train);

In [ ]:
# Make column of sale price on logrithmic scale
df_train['LogSalePrice'] = np.log1p(df_train['SalePrice'])

In [ ]:
# Transform data on log scale
sns.lmplot(x='Gr Liv Area', y='LogSalePrice', data=df_train);

In [ ]:
# Remove the two outlier buildings that are clearly not normal to our training set
df_train[df_train['Gr Liv Area'] > 4000]

In [ ]:
df_train = df_train[df_train['Gr Liv Area'] < 4000]

In [ ]:
# View other top correlate features
sns.lmplot(x='Mas Vnr Area', y='LogSalePrice', data=df_train);

In [ ]:
# After the scale has changed, there are two more outlier on the low price side, let's take a look 
df_train[df_train['LogSalePrice'] < 10]

In [ ]:
# We will just remove them since one is agricultural
df_train = df_train[df_train['LogSalePrice'] > 10]

In [ ]:
# Covert ordinal categorical to values and numerical categories to strings
ordered_rating_qual = { "NA": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}

# Not all of them but only a select few columns I want to convert
qual_list = ['Exter Qual',
             'Exter Cond',
             'Bsmt Qual',
             'Kitchen Qual',
             'Fireplace Qu',
             'Garage Qual'
            ]

for col in qual_list:
    df_train[col] = df_train[col].map(ordered_rating_qual)
    
    

#FUTURE REFERENCE METHOD FOR MAKING CATEGORICAL DICTIONARIES for .replace
#labels = df_train['Exter Qual'].astype('category').cat.categories.tolist()
#replace_map_comp = {'col' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

In [ ]:
for col in qual_list:
    df_test[col] = df_test[col].map(ordered_rating_qual)

In [ ]:
df_train['MS SubClass'] = df_train['MS SubClass'].astype(str)
df_train['MS SubClass'].dtype

In [ ]:
# View counts by categories to see if there is enough 
df_train['MS Zoning'].value_counts()

In [ ]:
# Just a reference list of items that didn't have enough split on categocial info
skip_list = ['Street', 'Utilities', 'Heating', 'Misc Feature']

In [ ]:
#Setup dummies on short list, doing all will not be helpful
dummy_list = ['Neighborhood',
              'Bldg Type',
             'House Style',
             'Roof Style',
             'Exterior 1st',
             'Exterior 2nd',
             'Foundation',
             'Functional',
             'Garage Type',
             'Garage Finish',
              'Paved Drive']

In [ ]:
# We know that doing get dummies on the test set will short change us on the columns so lets prep for that
# DNR AGAIN
for col in dummy_list:
    df_test[col] = df_test[col].fillna("NA")
    test_values = sorted(list(df_test[col].unique()))
    train_values = sorted(list(df_train[col].unique()))
    categories = set(train_values + test_values)
    df_test[col] = pd.Categorical(df_test[col], categories=categories)
    df_train[col] = pd.Categorical(df_train[col], categories=categories)

In [ ]:
# Create dummies on train dataset
df_dud = df_train

for col in dummy_list:
    dummy = pd.get_dummies(df_train[col], columns=col, prefix=col)
    df_dud = pd.concat([df_dud, dummy], axis=1)
    
df_dud.head()
    


In [ ]:
# Need to apply to test data too so columns match
df_test_dud = df_test

for col in dummy_list:
    dummy = pd.get_dummies(df_test[col], columns=col, prefix=col)
    df_test_dud = pd.concat([df_test_dud, dummy], axis=1)


In [ ]:
df_test_dud.shape

In [ ]:
df_dud.shape

In [ ]:
# Isolate correlation with respect to price with new data
corr_col_2 = df_dud.corrwith(df_train['SalePrice']).sort_values(ascending=False)[2:].to_frame()
fig2, ax2 = plt.subplots(figsize = (10,16))
corrplot = sns.heatmap(corr_col_2.head(20), cmap= 'coolwarm', annot=True, vmin=-1, vmax=1);

In [ ]:
#type(df_year)
sns.barplot(x=index, y='Mean Sale Price by mo_yr', data=df_year);

In [ ]:
df_year

### Create Features and Target

In [ ]:
#Fill in test numerical data for now

df_test_junk = pd.DataFrame()
for feature in num_columns:
    df_test_junk[feature] = df_test[feature].fillna(0)

In [ ]:
# Create matrices
num_columns = list(df_train._get_numeric_data().drop(['SalePrice', 'LogSalePrice'], axis=1).columns)
cat_columns = list(df_train.select_dtypes(include=['object']).columns)

In [ ]:
# Scale the data
ss = StandardScaler() # Instantiate Standard Scaler
df_scaled = ss.fit_transform(df_train[num_columns]) 
df_test_scaled = ss.fit_transform(df_test_junk[num_columns])

In [ ]:
# Print shapes to confirm
df_scaled.shape

In [ ]:
df_test_scaled.shape

In [ ]:
#NEW STEPS AFTER Dummies
num_columns_2 = list(df_dud._get_numeric_data().drop(['SalePrice', 'LogSalePrice'], axis=1).columns)
cat_columns_2 = list(df_dud.select_dtypes(include=['object']).columns)

for feature in num_columns:
    df_test_dud[feature] = df_test_dud[feature].fillna(0)

In [ ]:
# Scale the data
ss = StandardScaler() # Instantiate Standard Scaler
df_scaled = ss.fit_transform(df_dud[num_columns]) 
df_test_scaled = ss.fit_transform(df_test_dud[num_columns])

## Tools ands Functions

In [ ]:
import sklearn.metrics as metrics

In [ ]:
def metrics_dump(y,yhat):
    var_score = metrics.explained_variance_score(y,yhat)
    mae = metrics.mean_absolute_error(y,yhat)
    mse = metrics.mean_squared_error(y, yhat)
    msle = metrics.mean_squared_log_error(y, yhat)
    medae = metrics.median_absolute_error(y, yhat)
    r2_score = metrics.r2_score(y, yhat)
    results = {"explained_variance_score": var_score,
                            "mean_absolute_error": mae,
                            "mean_squared_error": mse,
                            "mean_squared_log_error": msle,
                            "median_absolute_error": medae,
                            "r2_score": r2_score}
    return results

### Regressions

In [ ]:
# Elastic Net 1 - A New Regression
enet_alphas = np.arange(0.01, 1.0, 0.005)
enet_ratio = 0.5
enet_model = ElasticNetCV(alphas=enet_alphas, l1_ratio=enet_ratio, cv=5)
enet_model = enet_model.fit(df_scaled, df_train['SalePrice'])

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(enet_model, df_scaled, df_train['SalePrice'], cv=10).mean()

In [ ]:
predictions_enet = enet_model.predict(df_test_junk[num_columns])

In [ ]:
# Elastic Net 2 - The GridSearch Strikes Back
from sklearn.metrics import mean_squared_error

# Create train and test sets
X = df_scaled
y = df_train['LogSalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state=42)

# Create the hyperparameter grid
l1_space = np.linspace(0, 1, 20)
alphas = np.logspace(0, 5, 100)
param_grid = {'l1_ratio': l1_space,
             'alpha': alphas}

# Instantiate the ElasticNet regressor: elastic_net
elastic_net = ElasticNet()

# Setup the GridSearchCV object: gm_cv
gs = GridSearchCV(elastic_net, param_grid, cv=5)

# Fit it to the training data
gs_results = gs.fit(X_train, y_train)

# Predict on the test set and compute metrics
y_pred = gs_results.predict(X_test)
r2 = gs_results.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)

gs_results = gs.fit(X_train, y_train)
print("l1 ratio: {}".format(gs_results.best_params_))
print("R squared: {}".format(r2))
print("MSE: {}".format(mse))

###### Iteration 1
Setup Basic Elastic Net using numerical feature

RESULTS: Iteration 1 high high MSE, need to optimize hyperparameters

Will try again with GridSearch

##### Iteration 2
Using Grid Search

RESULTS: Iteration 2 did not do better and actually scored worse than the non Grid search version.

    l1 ratio: {'alpha': 1.5885651294280527, 'l1_ratio': 0.24137931034482757}
    R squared: 0.7629059164812311
    MSE: 1508666865.9788153

Will try again with Log scaled SalePrice

##### Iteration 3
Using Grid Search and LogSalePrice
RESULTS: Iteration 3 did much better on R2, 

    l1 ratio: {'alpha': 1.0, 'l1_ratio': 0.0}
    R squared: 0.8467819092554696
    MSE: 0.025730495020561128 (Error is not one for one since we transformed the target)

Will try again with dummies on select categorical columns and updating categorical with ordinal direction to scale

##### Iteration 4
Using Grid Search and LogSalePrice and Dummies added and Ordinal Update
RESULTS: Iteration 4 did much better on R^2, and we reduced our error by a substantial amount

    l1 ratio: {'alpha': 1.0, 'l1_ratio': 0.0}
    R squared: 0.8860361852037677
    MSE: 0.01826427918929721

Will try again with .....

In [ ]:
# Random Forest Run 1 - test data input as compared to other Kaggle submissions and where we could be heading
from sklearn.ensemble import RandomForestRegressor

In [ ]:
params = {}

gs = GridSearchCV(RandomForestRegressor(), param_grid= params)
gs.fit(df_train[num_columns], df_train['SalePrice'])
print(gs.best_score_)
print(gs.best_params_)

## CSV Creation and Kaggle Submission

In [ ]:
########## All interiations will be done above and this section will be for creating submissions only ##########
#Set Predictions to be added to test data for Kaggle
#predictions_enet = enet_model.predict(df_test_junk[num_columns])
predictions_gs = gs_results.predict(df_test_scaled)

In [ ]:
predictions_from_log = np.expm1(predictions_gs)

In [ ]:
df_test_junk['SalePrice'] = predictions_from_log

In [ ]:
df_test_junk['Id'] = df_test['Id']

In [ ]:
# Create CSV for Kaggle submission
df_test_junk[['Id', 'SalePrice']].to_csv("submission_loggridsearchdummies.csv", index=False)